<center>
<img src="https://drive.google.com/uc?export=download&id=1nv5uGKO9BLD9Y19LnZZH35nnQghZsPdD" />

# Feature Engineering and Feature Selection

Para empezar, vamos a revisar tres tareas similares pero diferentes: 

* **feature extraction** and **feature engineering**: Transformación de data(raw) en características adecuadas para modelado;
* **feature transformation**: Transformación de data para mejorar la precisión de los algoritmos;
* **feature selection**: Removiendo características innecesarias.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Temas

1. [Feature Extraction](#1.-Feature-Extraction)
 - [Texts](#Texts)
 - [Geospatial data](#Geospatial-data)
 - [Date and time](#Date-and-time)
 - [Time series, web, etc.](#Time-series,-web,-etc.)

## 1. Feature Extraction


En la práctica, los datos rara vez se presentan en forma de matrices listas para usar. Es por eso que cada tarea comienza con la extracción de características. A veces, puede ser suficiente leer el archivo .csv y convertirlo en `numpy.array`, pero esta es una rara excepción. Veamos algunos de los tipos populares de datos de los que se pueden extraer características.

In [ ]:
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Texts

El texto es un tipo de datos que puede venir en diferentes formatos, revisaremos los más populares.

Antes de trabajar con texto, hay que tokenizarlo. La tokenización implica dividir el texto en unidades (tokens). Los tokens son sólo las palabras. Pero el dividir por palabras nos puede llevar a perder parte del significado-- "Santa Bárbara" es un token, no dos, pero "rock'n'roll" no debe dividirse en dos token. Hay tokenizadores listos para usar que tienen en cuenta las peculiaridades del lenguaje, pero también cometen errores, especialmente cuando trabajas con fuentes de texto específicas (periódicos, jerga, errores ortográficos, errores tipográficos).

Después de la tokenización, normalizamos los datos. Para el texto, se trata de la derivación y/o lematización; Estos son procesos similares utilizados para procesar diferentes formas de una palabra. Se puede leer sobre la diferencia entre ellos [aqui](http://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html).
Entonces, ahora que hemos convertido el documento en una secuencia de palabras, podemos representarlo con vectores. El enfoque más fácil se llama Bag of Words: creamos un vector con la longitud del diccionario, calculamos el número de ocurrencias de cada palabra en el texto y colocamos ese número de ocurrencias en la posición apropiada en el vector. El proceso descrito parece más simple en el código:

In [ ]:
from functools import reduce 
import numpy as np

# definicion de corpus
texts = [['i', 'have', 'a', 'cat'], 
        ['he', 'have', 'a', 'dog'], 
        ['he', 'and', 'i', 'have', 'a', 'cat', 'and', 'a', 'dog']]

dictionary = list(enumerate(set(list(reduce(lambda x, y: x + y, texts)))))
print(dictionary)
def vectorize(text): 
    vector = np.zeros(len(dictionary)) 
    for i, word in dictionary: 
        num = 0 
        for w in text: 
            if w == word: 
                num += 1 
        if num: 
            vector[i] = num 
    return vector

for t in texts: 
    print(vectorize(t))

Esta es una ilustración del proceso:
<img src="https://drive.google.com/uc?export=download&id=18dEqfTKT10i5_EJz4MLy_hywpQ_IusUJ" />

Esta es una implementación extremadamente ingenua. En la práctica, debe considerar palabras de parada, la longitud máxima del diccionario, estructuras de datos más eficientes (generalmente los datos de texto se convierten en un matrices esparsa), etc.

Cuando utilizamos algoritmos como Bag of Words, perdemos el orden de las palabras en el texto, lo que significa que los textos "i have no cows" y "no, i have cows" aparecerán idénticos después de la vectorización cuando, de hecho, tienen el significado opuesto. Para evitar este problema, podemos volver a visitar nuestro paso de tokenización y usar N-grams (la *secuencia* de N tokens consecutivos) en su lugar.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1))
vect.fit_transform(['i have no cows','no, i have cows']).toarray()

In [ ]:
vect.vocabulary_

In [ ]:
vect = CountVectorizer(ngram_range=(1,2))
vect.fit_transform(['i have no cows','no, i have cows']).toarray()

In [ ]:
vect.vocabulary_

También tenga en cuenta que uno no tiene que usar sólo palabras. En algunos casos, es posible generar N-gram de caracteres. Este enfoque podría dar cuenta de la similitud de palabras relacionadas o manejar errores tipográficos.

In [ ]:
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(3,3), analyzer='char_wb')

n1, n2, n3, n4 = vect.fit_transform(['andersen', 'petersen', 'petrov', 'smith']).toarray()


euclidean(n1, n2), euclidean(n2, n3), euclidean(n3, n4)

Agregando a la idea de Bag of Words: las palabras que rara vez se encuentran en el corpus (en todos los documentos del dataset) pero que están presentes en un documento en particular podrían ser más importantes. Entonces tiene sentido aumentar el peso de más palabras específicas del dominio para separarlas de las palabras comunes. Este enfoque se llama TF-IDF (term frequency-inverse document frequency), que no se puede escribir en unas pocas líneas, por lo que debe consultar los detalles en referencias como [wiki](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). La opción predeterminada es la siguiente:

<img src="https://drive.google.com/uc?export=download&id=1zRnAL7xslzRl3odfsLa3yzbSR9SMw0CM" />

Usando estos algoritmos, es posible obtener una solución para un problema simple, que puede servir como línea base. Sin embargo, para aquellos a quienes no les gustan los clásicos, hay nuevos enfoques. Un método popular es Word2Vec, pero también hay algunas alternativas (GloVe, Fasttext, etc.).

Word2Vec es un caso especial de los algoritmos word embedding. Usando Word2Vec y modelos similares, no sólo podemos vectorizar palabras en un espacio de alta dimensión (típicamente unos pocos cientos de dimensiones) sino también comparar su similitud semántica. Este es un ejemplo clásico de operaciones que se pueden realizar en conceptos vectorizados: **king - man + woman = queen.**

![image](https://cdn-images-1.medium.com/max/800/1*K5X4N-MJKt8FGFtrTHwidg.gif)

Vale la pena señalar que este modelo no comprende el significado de las palabras, simplemente trata de posicionar los vectores de manera que las palabras utilizadas en el contexto común estén cerca unas de otras.

Dichos modelos necesitan ser entrenados en data sets muy grandes para que las coordenadas del vector capturen la semántica. Se puede descargar un modelo previamente entrenado para sus propias tareas
[aquí](https://github.com/3Top/word2vec-api#where-to-get-a-pretrained-models).

### **Ejercicio: Clasificación de Spam usando Support Vector Machines.**
#### CONTEXTO:    SMS Spam Collection es un conjunto de mensajes etiquetados SMS que se han recopilado para la investigación de SMS Spam. Contiene un conjunto de mensajes SMS en inglés 5,574 mensajes etiquetados de acuerdo a su contenido 'ham' (legítimo) o 'spam'.

AGRADECIMIENTO:  El dataset original se puede encontrar [aquí](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection). Los creadores desean tener en cuenta que en caso de que encuentre útil el conjunto de datos, haga referencia al documento anterior y la página web: http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

CONTENIDO: Los archivos contienen un mensaje por línea. Cada línea está compuesta por dos columnas: v1 contiene la etiqueta (ham o spam) y v2 contiene el texto sin formato. Este corpus se ha recopilado de forma gratuita.

### **Librerias**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### **Explorando el Dataset**

In [ ]:
data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
data.head(n=10)

### **Análisis de texto**
1.  Graficar y encontrar la frecuencia de las palabras en los mensajes spam y no-spam(ham)
2.  Describir cada gráfico

In [ ]:
data= pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',encoding='latin-1')
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"Tipo", "v2":"Letras"})
data.head()

In [ ]:
data.info()

In [ ]:
count1=Counter(" ".join(data[data['Tipo']=='ham']["Letras"]).split()).most_common(30)
df1=pd.DataFrame.from_dict(count1)
print(df1.head())
df1 = df1.rename(columns={0:"palabras non-spam", 1 :"count"})
count2 = Counter(" ".join(data[data['Tipo']=='spam']["Letras"]).split()).most_common(30)
df2 = pd.DataFrame.from_dict(count2)
df2 = df2.rename(columns={0:"palabras spam", 1:"count_"})

In [ ]:
df1.plot.bar(legend="False")
y_pos=np.arange(len(df1["palabras non-spam"]))
plt.xticks(y_pos,df1["palabras non-spam"])
plt.title('Palabras frecuentes en mensajes no-spam')
plt.xlabel('Palabras')
plt.ylabel('Numero')
plt.show()

Las palabras con mayor frecuencia que se presentan en los mensajes no spam  son :to,you,I,the,a,and,i,in,u,is,entre otras palabras que son minoria.

In [ ]:
df2.plot.bar(legend= False , color='red')
y_pos=np.arange(len(df2["palabras spam"]))
plt.xticks(y_pos,df2["palabras spam"])
plt.title('Palabras frecuentes en mensajes spam')
plt.xlabel('Palabras')
plt.ylabel('Numero')
plt.show()

Las palabras mas frecuentes en los mensajes spam son:to,a,your,call,or,the,2,for,you,is,Call,entre otras en minoria.


In [ ]:
data['Tipo'].value_counts()

In [ ]:
data['Tipo'].value_counts().plot.bar()

De un total de 5572 mensajes ,4825 son mensajes valioso y 747 son mensajes de publicidad o sin importancia.

In [ ]:
data["Tipo"].value_counts().plot(kind = 'pie', explode = [0, 0.1], figsize = (6, 6), autopct = '%1.1f%%', shadow = True)
plt.ylabel("Spam vs Ham")
plt.legend(["Ham", "Spam"])
plt.show()

El 86.6% de los mensajes que llegan a la bandeja de entrada son mensajes con importancia y  un 13.4% son mensajes sin importancia o de propraganda.

In [ ]:
data['Tipo'].replace('spam', 0, inplace = True)
data['Tipo'].replace('ham', 1, inplace = True)

# checking the values of the labels now
data['Tipo'].value_counts()

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(background_color = 'gray', width = 1000, height = 1000, max_words = 50).generate(str(data['Letras']))

plt.rcParams['figure.figsize'] = (10, 10)
plt.title('Most Common words in the dataset', fontsize = 20)
plt.axis('off')
plt.imshow(wordcloud)

Las palabras mas populares que podemos encontrar en los mensajes que nos llegan al correo son:name,say,crazy,jurong,go,point,ok,availables, entre otras.
    

In [ ]:
spam = ' '.join(text for text in data['Letras'][data['Tipo'] == 0])

wordcloud = WordCloud(background_color ='white', max_words = 50, height = 1000, width = 1000).generate(spam)

plt.rcParams['figure.figsize'] = (10, 10)
plt.axis('off')
plt.title('Most Common Words in Spam Messages', fontsize = 20)
plt.imshow(wordcloud)

Las palabras mas comunes o populares que podemos encontrar en los mensajes spam son:Free,text,now,call,now,txt,messge,150p,reply,entre otras.

In [ ]:
ham = ' '.join(text for text in data['Letras'][data['Tipo'] == 1])

wordcloud = WordCloud(background_color = 'gray', max_words = 50, height = 1000, width = 1000).generate(ham)

plt.rcParams['figure.figsize'] = (10, 10)
plt.axis('off')
plt.title('Most Common Words in Ham Messages', fontsize = 20)
plt.imshow(wordcloud)

Las palabras mas comunes o populares que encontramos en los mensajes no spam son:will,It,gt,got,time,need,ok,now,entre otras.

### **feature extraction and feature engineering **

1. Preprocesamiento de texto 
2. Creación de tokens y el filtrado de palabras clave 
(puede usar un componente de alto nivel como: CountVectorizer que puede crear un diccionario de características y transformar documentos en vectores de características.)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [ ]:
feat= feature_extraction.text.CountVectorizer(stop_words = 'english',max_features=100)
X= feat.fit_transform(data["Letras"])
np.shape(X)

In [ ]:
data['Tipo'].replace('spam', 0, inplace = True)
data['Tipo'].replace('ham', 1, inplace = True)
data.head()

In [ ]:
X_train,X_test,y_train,y_test = model_selection.train_test_split(X, data["Tipo"], test_size=0.2, random_state=42)
print([np.shape(X_train),np.shape(X_test)])
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


### **Análisis Predictivo**

1. El objetivo es predecir si un nuevo sms es spam o no-spam. usando SVM
2. validar: Matriz de confusión

In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.svm import SVC

In [ ]:
# usamos Support Vector Machine de :https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
import string
string.punctuation

In [ ]:
from nltk.corpus import stopwords
stopwords.words("english")[100:110]

In [ ]:
from sklearn import svm

In [ ]:
svc = svm.SVC()
svc.fit(x_train, y_train)
score_train = svc.score(X_train, y_train)
score_test = svc.score(X_test, y_test)
SVC()

In [ ]:
# para validar debe usar una matriz de confusión usando el siguiente código:
matr_confusion_test = metrics.confusion_matrix(y_test, svc.predict(x_test))
pd.DataFrame(data = matr_confusion_test, columns = ['Prediccion spam', 'Prediccion no-spam'],
            index = ['Real spam', 'Real no-spam'])